In [ ]:
!pwd

In [ ]:
import numpy as np
import torch
from torch import Tensor, nn, optim
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.transforms.functional import pad
from torch.utils.data import DataLoader
from multiprocessing import cpu_count

from thesisproject.data.utils import get_slice_loaders
from thesisproject.models import UNet
from thesisproject.train import training_loop

In [ ]:
class SquarePad:
    def __call__(self, image: Tensor):
        imsize = image.shape
        max_edge = np.argmax(imsize)
        pad_amounts = [imsize[max_edge] - imsize[0], imsize[max_edge] - imsize[1], imsize[max_edge] - imsize[2]]

        padding = [int(np.floor(pad_amounts[0] / 2)),
                   int(np.ceil(pad_amounts[0] / 2)),
                   int(np.floor(pad_amounts[1] / 2)),
                   int(np.ceil(pad_amounts[1] / 2)),
                   int(np.floor(pad_amounts[2] / 2)),
                   int(np.ceil(pad_amounts[2] / 2)),] #left, right, top, bottom, front, back
        padding = tuple(padding[::-1])

        padded_im = F.pad(image, padding, "constant", 0)
        return padded_im

class Standardize:
    def __call__(self, image: Tensor):
        """
        Normalize image to 0 mean and standard deviation 1
        """
        
        std, mean = torch.std_mean(image)
        return (image - mean) / std

In [ ]:
path = "../knee_data/"
volume_transform = T.Compose([SquarePad(), Standardize()])
label_transform = SquarePad()

train_data, val_data = get_slice_loaders(path, volume_transform=volume_transform, label_transform=label_transform)

train_loader = DataLoader(train_data, batch_size=8, num_workers=4)
val_loader = DataLoader(val_data, batch_size=8, num_workers=4)

## Train
label_keys = ["Lateral femoral cart.",
                "Lateral meniscus",
                "Lateral tibial cart.",
                "Medial femoral cartilage",
                "Medial meniscus",
                "Medial tibial cart.",
                "Patellar cart.",
                "Tibia"]
net = UNet(1, 9, 384, class_names=label_keys)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=5e-5)

In [ ]:
training_loop(
    net,
    criterion,
    optimizer,
    train_loader,
    val_loader,
    num_epochs=100,
    cont=True
)